<p style="float:right">
<img src="../images/logos/cu.png" style="display:inline" />
<img src="../images/logos/cires.png" style="display:inline" />
<img src="../images/logos/nasa.png" style="display:inline" />
<img src="../images/logos/nsidc_daac.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 5 Extra

## ???????????

Let's use what we know about our source data to write a couple of routines to compute anomalies and display them on a map.

First, some setup:

In [ ]:
%%bash
rm -f nhsce_v01r01_19661004_20160201.nc
unzip $PWD/../data/nhsce_v01r01_19661004_20160201.nc.zip

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

snowcover_file = 'nhsce_v01r01_19661004_20160201.nc'
dataset = xr.open_dataset(snowcover_file)

We'll use the [`DataSet.groupby`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.groupby.html) function to gather groups of time dimensions. We'll group all of the time index values by the month number.  Accessing the `groups` attribute returns a dictionary where the keys are months and the values are a list of indices into the dataset time index for that month.

In [ ]:
month_indices = dataset.groupby('time.month').groups

print("Keys:", month_indices.keys(), "<-- One for each month")
print("Feburary Indices:", month_indices[2][0:5], "...")

We can use a list of indices to select data with `isel`:

In [ ]:
month_num = 6

weeks = dataset['time'].isel(time=month_indices[month_num])

print("first 10 dataSet['time'] Values:\n ", weeks.values[0:10])
print("\nTotal number of elements in month_indices:", len(weeks))

We'll use every available measurement in the `DataSet` to compute a median snow cover for a given month.

We'll do this by computing a mean across time for each of the month's samples. This gives us a fracional probability of any measurement having snow cover. By taking those values that are greater than or equal to .5, we get a median snow cover for the month.

Choose a month with some snow and compute median snow cover for that month:

In [ ]:
month_number = 2
average_snowcover = dataset['snow_cover_extent'].isel(time=month_indices[month_number]).mean(dim='time')
median_snowcover = average_snowcover > .5

Extract our latitude, longitude and land variables:

In [ ]:
lats = dataset.latitude.values
lons = dataset.longitude.values
land = dataset.land.values

Create a function that will return a categorical grid of the snow cover anomaly. For each cell, we want to identify whether it is part of the selected month's extent and/or median extent, or if it is land or ocean.

In [ ]:
def anomaly_snowcover(selected_snow_cover, median_snowcover, land):

    sel = selected_snow_cover.values.astype(bool)
    med = median_snowcover.values.astype(bool)
    land = land.astype(bool)

    # Do logical intersections of the data
    both     =  sel &  med
    only_med = ~sel &  med
    only_sel =  sel & ~med

    # Assign a those intersections values.
    out = np.zeros_like(land.astype(int))
    out[~land] = 0
    out[land] = 1
    out[both] = 2
    out[only_med] = 3
    out[only_sel] = 4

    return out

Create a [colormap](http://matplotlib.org/api/colors_api.html?highlight=listedcolormap#matplotlib.colors.ListedColormap) and [normalizer](http://matplotlib.org/users/colormapnorms.html) for plotting the `anomaly_snowcover` output, knowing our data will have only values 0 through 4:

In [ ]:
# Choose some nice colors
categorical_cmap = mpl.colors.ListedColormap(colors=['#D4EFFA', '#A3BAA5','#FEFEFE','#BC80BC', '#ACD665'])

# center your bounds around your datapoint.
bounds = [-.5, .5, 1.5, 2.5, 3.5, 4.5]
norm = mpl.colors.BoundaryNorm(bounds, categorical_cmap.N)

Import some other libraries we'll need:

In [ ]:
from mpl_toolkits.basemap import Basemap
from ipywidgets import interact
import ipywidgets as widgets

Now use a widget to plot anomalies of snow cover:

In [ ]:
import datetime as dt
import calendar as cal

def title_function(datetime64ns):
    date = datetime64ns.astype('M8[ms]').astype(dt.date)
    return "Snow Cover: {0}-{1} compared to median".format(cal.month_abbr[date.month], date.year)

def selected_month_label(datetime64ns):
    date = datetime64ns.astype('M8[ms]').astype(dt.date)
    return '{0} {1} Only'.format(cal.month_abbr[date.month], date.year)

@interact(index_in=widgets.IntSlider(min=0,
                                     max=len(month_indices[month_number])-1,
                                     step=1,
                                     value=0,
                                     continuous_update=False))

def plot_anomaly(index_in=0):
    index = month_indices[month_number][index_in]
    plt.figure(figsize=(10, 10))
    m = Basemap(projection='npstere', boundinglat=30, lon_0=-45)
    m.drawcoastlines()

    parallels = np.arange(0, 90, 20)
    m.drawparallels(parallels, labels=[True])
    meridians = np.arange(-180, 180, 45)
    m.drawmeridians(meridians, labels=[True, True, False, True, True, True, True, True])
    
    the_data = anomaly_snowcover(dataset['snow_cover_extent'].isel(time=index), median_snowcover, land)
    m.pcolor(lons, lats, the_data, latlon=True, cmap=categorical_cmap, norm=norm)
    
    times = dataset['time'].isel(time=index).values
    
    cbar = plt.colorbar(ticks=[0, 1, 2, 3, 4], norm=norm)
    cbar.set_ticklabels(['Ocean', 'Land', 'Both', 'Median Only', selected_month_label(times)])
    plt.title(title_function(times))
    plt.draw()